In [ ]:
import csv  # Import the csv module
import json
import pymongo
from bson.code import Code
import tkinter as tk
from tkinter import messagebox

# Read the dataset from CSV file
with open('heart.csv', mode='r') as file:
    # Assuming the first row contains column headers
    reader = csv.DictReader(file)
    data = [row for row in reader]

# Convert each record to JSON format
json_data = json.dumps(data, indent=4)

# Write the JSON data to a file
with open("dataset.json", "w") as json_file:
    json_file.write(json_data)

print("Dataset converted to JSON and saved as 'dataset.json'")

client = pymongo.MongoClient("mongodb://localhost:27017/")

db = client["heart"]

with open("dataset.json") as f:
    data = json.load(f)

for patient in data:
    db["patient"].insert_one({
        "Patient ID": patient["Patient ID"],
        "Age": patient["Age"],
        "Sex": patient["Sex"],
        "Country": patient["Country"],
        "Continent": patient["Continent"],
        "Hemisphere": patient["Hemisphere"]
    })
    db["health"].insert_one({
        "Patient ID": patient["Patient ID"],
        "Cholesterol": patient["Cholesterol"],
        "Blood Pressure": patient["Blood Pressure"],
        "Heart Rate": patient["Heart Rate"],
        "BMI": patient["BMI"],
        "Triglycerides": patient["Triglycerides"]
    })
    db["lifestyle"].insert_one({
        "Patient ID": patient["Patient ID"],
        "Smoking": patient["Smoking"],
        "Alcohol Consumption": patient["Alcohol Consumption"],
        "Exercise Hours Per Week": patient["Exercise Hours Per Week"],
        "Diet": patient["Diet"],
        "Sedentary Hours Per Day": patient["Sedentary Hours Per Day"],
        "Physical Activity Days Per Week": patient["Physical Activity Days Per Week"],
        "Sleep Hours Per Day": patient["Sleep Hours Per Day"]
    })
    db["healthHistory"].insert_one({
        "Patient ID": patient["Patient ID"],
        "Diabetes": patient["Diabetes"],
        "Family History": patient["Family History"],
        "Previous Heart Problems": patient["Previous Heart Problems"],
        "Medication Use": patient["Medication Use"],
        "Stress Level": patient["Stress Level"],
        "Income": patient["Income"],
        "Heart Attack Risk": patient["Heart Attack Risk"]
    })

print("Data has been inserted into the MongoDB database")


In [ ]:
################################################ ALL GUI ########################################################

In [ ]:
import tkinter as tk
from bson.code import Code
import pymongo

# Connect to MongoDB
client = pymongo.MongoClient("mongodb://localhost:27017/")
db = client["heart"]

# Main window
root = tk.Tk()
root.title("Main Menu")


#############################################  Patient  #################################################################
def open_patient_data_management():
    patient_window = tk.Toplevel(root)
    patient_window.title("Patient Data Management")

    def insert_patient_data():
        patient_id = patient_id_entry.get()
        # Check if patient ID already exists in the database
        existing_patient = db["patient"].find_one({"Patient ID": patient_id})
    
        if existing_patient:
            whiteboard.insert(tk.END, "Patient ID already exists. Please use a different ID.\n")
        else:
            try:
                patient_data = {
                    "Patient ID": patient_id,
                    "Age": int(age_entry.get()),
                    "Sex": sex_entry.get(),
                    "Country": country_entry.get(),
                    "Continent": continent_entry.get(),
                    "Hemisphere": hemisphere_entry.get()
                }
                db["patient"].insert_one(patient_data)
                whiteboard.insert(tk.END, "Patient data inserted successfully.\n")
            except ValueError:
                whiteboard.insert(tk.END, "Invalid input for Age. Please enter a valid number.\n")
            
    def read_patient_data():
        patient_id = patient_id_entry.get()
        patient = db["patient"].find_one({"Patient ID": patient_id})
        if patient:
            whiteboard.insert(tk.END, f"Patient Data: {str(patient)}\n")
        else:
            whiteboard.insert(tk.END, "Patient not found.\n")

    def update_patient_data():
        patient_id = patient_id_entry.get()
        patient = db["patient"].find_one({"Patient ID": patient_id})
        if patient:
            update_data = {
                "$set": {
                    "Age": int(age_entry.get()),
                    "Sex": sex_entry.get(),
                    "Country": country_entry.get(),
                    "Continent": continent_entry.get(),
                    "Hemisphere": hemisphere_entry.get()
                }
            }
            db["patient"].update_one({"Patient ID": patient_id}, update_data)
            whiteboard.insert(tk.END, "Patient data updated successfully.\n")
        else:
            whiteboard.insert(tk.END, "Patient not found.\n")

    def delete_patient_data():
        patient_id = patient_id_entry.get()
        result = db["patient"].delete_one({"Patient ID": patient_id})
        if result.deleted_count == 1:
            whiteboard.insert(tk.END, "Patient data deleted successfully.\n")
        else:
            whiteboard.insert(tk.END, "Patient not found.\n")

    def create_index_and_query():
        try:
            # Single Field Indexes for patient table
            field = "Age"
            index_name = f"{field}_index"
            index_info = db["patient"].index_information()
            
            if index_name not in index_info:
                db["patient"].create_index([(field, 1)], name=index_name)
                whiteboard.insert(tk.END, f"Index {index_name} created on field {field} in descending order.\n")
            else:
                whiteboard.insert(tk.END, f"Index {index_name} already exists on field {field}.\n")
    
            # Display index information
            # index_info = db["patient"].index_information()
            # whiteboard.insert(tk.END, "Indexes:\n")
            # for index_name, info in index_info.items():
            #     whiteboard.insert(tk.END, f"{index_name}: {info}\n")
    
            # Perform the query using .hint() to use the "Age" index
            results = db["patient"].find().hint({field: 1}).sort("Age", -1)
            whiteboard.insert(tk.END, "Query Results using hint on 'Age':\n")
            for result in results:
                whiteboard.insert(tk.END, f"{result}\n")
        except Exception as e:
            whiteboard.insert(tk.END, f"Error creating index or performing query with hint: {str(e)}\n")

    # Function to drop single-field indexes and display the result
    def drop_single_field_indexes():
        try:
            # Drop Single Field Indexes for patient table
            for field in ["Age"]:
                index_name = f"{field}_index"
                if index_name in db["patient"].index_information():
                    db["patient"].drop_index(index_name)
                    whiteboard.insert(tk.END, f"Index {index_name} dropped from field {field}.\n")
                else:
                    whiteboard.insert(tk.END, f"Index {index_name} not found on field {field}.\n")
        except Exception as e:
            whiteboard.insert(tk.END, f"Error dropping indexes: {str(e)}\n")
        
    # Function to count patients by sex and display the result
    def count_patients_by_sex():
        try:
            pipeline = [{"$group": {"_id": "$Sex", "count": {"$sum": 1}}}]
            result = list(db["patient"].aggregate(pipeline))
            whiteboard.insert(tk.END, "Patients Count by Sex:\n")
            for item in result:
                whiteboard.insert(tk.END, f"{item['_id']}: {item['count']}\n")
        except Exception as e:
            whiteboard.insert(tk.END, f"Error counting patients by sex: {str(e)}\n")

    # Function to calculate average age of patients and display the result
    def aggregate_patients_by_age():
        try:
            pipeline = [
                {"$group": {"_id": "$Age", "count": {"$sum": 1}}},
                {"$sort": {"_id": 1}}
            ]
            result = list(db["patient"].aggregate(pipeline))
            whiteboard.insert(tk.END, "Patients Count by Age:\n")
            for item in result:
                whiteboard.insert(tk.END, f"Age: {item['_id']}, Count: {item['count']}\n")
        except Exception as e:
            whiteboard.insert(tk.END, f"Error aggregating patients by age: {str(e)}\n")

    def select_male_patients():
        try:
            pipeline = [{"$match": {"Sex": "Male"}}]
            result = list(db["patient"].aggregate(pipeline))
            whiteboard.insert(tk.END, "Male Patients:\n")
            for patient in result:
                whiteboard.insert(tk.END, f"{patient}\n")
        except Exception as e:
            whiteboard.insert(tk.END, f"Error selecting male patients: {str(e)}\n")

    # Function to perform map-reduce operation to sum FM (assuming "sex" field represents gender)
    def map_reduce_sum_FM():
        try:
            # Define the map function
            mapper = Code("""
                function () {
                    if (this.Sex) {
                        emit(this.Sex, 1);
                    }
                }
            """)
            # Define the reduce function
            reducer = Code("""
                function (key, values) {
                    return Array.sum(values);
                }
            """)
    
            # Execute the MapReduce operation
            result = db.command('mapReduce', "patient", map=mapper, reduce=reducer, out={'inline': 1})
    
            # Print results
            whiteboard.insert(tk.END, "MapReduce Results:\n")
            if result['results']:
                for doc in result['results']:
                    whiteboard.insert(tk.END, f"Sex: {doc['_id']}, Count: {doc['value']}\n")
            else:
                whiteboard.insert(tk.END, "No results found. Please check if the 'Sex' field exists and has valid values.\n")
        except Exception as e:
            whiteboard.insert(tk.END, f"Error performing MapReduce operation: {str(e)}\n")

    def map_reduce_country():
        try:
            # Define the map function
            map_fun = Code("""
                function() {
                    emit(this.Country, 1);
                }
            """)

            # Define the reduce function
            reduce_fun = Code("""
                function(key, values) {
                    return Array.sum(values);
                }
            """)
    
            # Execute the MapReduce operation
            result = db.command({
                "mapReduce": "patient",
                "map": map_fun,
                "reduce": reduce_fun,
                "out": {"inline": 1}
            })
            # Display the result in the whiteboard
            whiteboard.insert(tk.END, "MapReduce results on Country:\n")
            for doc in result['results']:
                whiteboard.insert(tk.END, f"{doc['_id']}: {doc['value']}\n")
        except Exception as e:
            whiteboard.insert(tk.END, f"Error performing MapReduce: {str(e)}\n")  

    
    def go_back():
        patient_window.destroy()
        root.deiconify()

    # insert_button = tk.Button(patient_window, text="Insert", command=insert_patient_data)
    # insert_button.grid(row=6, column=0, padx=5, pady=5)
    # Patient Data Management widgets and functions
    # Insert your existing Patient Data Management code here, for example:
    tk.Label(patient_window, text="Patient ID:").grid(row=0, column=0, padx=5, pady=5)
    tk.Label(patient_window, text="Age:").grid(row=1, column=0, padx=5, pady=5)
    tk.Label(patient_window, text="Sex (Male/Female):").grid(row=2, column=0, padx=5, pady=5)
    tk.Label(patient_window, text="Country:").grid(row=3, column=0, padx=5, pady=5)
    tk.Label(patient_window, text="Continent:").grid(row=4, column=0, padx=5, pady=5)
    tk.Label(patient_window, text="Hemisphere (Northern/Southern):").grid(row=5, column=0, padx=5, pady=5)

    patient_id_entry = tk.Entry(patient_window)
    patient_id_entry.grid(row=0, column=1, padx=5, pady=5)
    age_entry = tk.Entry(patient_window)
    age_entry.grid(row=1, column=1, padx=5, pady=5)
    sex_entry = tk.Entry(patient_window)
    sex_entry.grid(row=2, column=1, padx=5, pady=5)
    country_entry = tk.Entry(patient_window)
    country_entry.grid(row=3, column=1, padx=5, pady=5)
    continent_entry = tk.Entry(patient_window)
    continent_entry.grid(row=4, column=1, padx=5, pady=5)
    hemisphere_entry = tk.Entry(patient_window)
    hemisphere_entry.grid(row=5, column=1, padx=5, pady=5)

    insert_button = tk.Button(patient_window, text="Insert", command=lambda: [insert_patient_data(), insert_button.config(bg="blue", fg="white")])
    insert_button.grid(row=6, column=0, padx=5, pady=5)

    read_button = tk.Button(patient_window, text="Read", command=lambda: [read_patient_data(), read_button.config(bg="blue", fg="white")])
    read_button.grid(row=6, column=1, padx=5, pady=5)

    update_button = tk.Button(patient_window, text="Update", command=lambda: [update_patient_data(), update_button.config(bg="blue", fg="white")])
    update_button.grid(row=7, column=0, padx=5, pady=5)

    delete_button = tk.Button(patient_window, text="Delete", command=lambda: [delete_patient_data(), delete_button.config(bg="blue", fg="white")])
    delete_button.grid(row=7, column=1, padx=5, pady=5)
    
    index_button = tk.Button(patient_window, text="Create Indexes", command=lambda: [create_index_and_query(), index_button.config(bg="blue", fg="white")])
    index_button.grid(row=8, column=0, padx=5, pady=5)
    
    drop_index_button = tk.Button(patient_window, text="Drop Indexes", command=lambda: [drop_single_field_indexes(), drop_index_button.config(bg="blue", fg="white")])
    drop_index_button.grid(row=8, column=1, padx=5, pady=5)
    
    count_sex_button = tk.Button(patient_window, text="Count Patients by Sex", command=lambda: [count_patients_by_sex(), count_sex_button.config(bg="blue", fg="white")])
    count_sex_button.grid(row=9, column=0, padx=5, pady=5)
    
    age_button = tk.Button(patient_window, text="Aggregate Patients by Age", command=lambda: [aggregate_patients_by_age(), age_button.config(bg="blue", fg="white")])
    age_button.grid(row=9, column=1, padx=5, pady=5)
    
    male_patients_button = tk.Button(patient_window, text="Select Male Patients", command=lambda: [select_male_patients(), male_patients_button.config(bg="blue", fg="white")])
    male_patients_button.grid(row=10, column=0, padx=5, pady=5)
    
    map_reduce_sum_button = tk.Button(patient_window, text="MapReduce Sum FM", command=lambda: [map_reduce_sum_FM(), map_reduce_sum_button.config(bg="blue", fg="white")])
    map_reduce_sum_button.grid(row=10, column=1, padx=5, pady=5)
    
    map_reduce_country_button = tk.Button(patient_window, text="MapReduce Country", command=lambda: [map_reduce_country(), map_reduce_country_button.config(bg="blue", fg="white")])
    map_reduce_country_button.grid(row=11, column=0, padx=5, pady=5)

    back_button = tk.Button(patient_window, text="Back to Main Menu", command=go_back)
    back_button.grid(row=12, column=0, columnspan=2, pady=10)

    whiteboard_frame = tk.Frame(patient_window)
    whiteboard_frame.grid(row=0, column=2, rowspan=11, padx=20, pady=5, sticky=tk.N+tk.S)

    whiteboard_scrollbar = tk.Scrollbar(whiteboard_frame)
    whiteboard_scrollbar.pack(side=tk.RIGHT, fill=tk.Y)

    global whiteboard
    whiteboard = tk.Text(whiteboard_frame, height=30, width=100, yscrollcommand=whiteboard_scrollbar.set)
    whiteboard.pack(side=tk.LEFT, fill=tk.BOTH)
    whiteboard_scrollbar.config(command=whiteboard.yview)

    root.withdraw()
    patient_window.mainloop()


#############################################  Health  #################################################################
def open_health_data_management():
    health_window = tk.Toplevel(root)
    health_window.title("Health Data Management")


    def insert_health_data():
        patient_id = patient_id_entry.get()
    
        # Check if patient ID exists in the patient collection
        existing_patient = db["health"].find_one({"Patient ID": patient_id})
    
        if existing_patient:
            whiteboard.insert(tk.END, "Patient ID already exists. Please use a different ID.\n")
        else:
            try:
                health_data = {
                    "Patient ID": patient_id,
                    "Cholesterol": int(cholesterol_entry.get()),
                    "Blood Pressure": {
                        "Systolic": int(systolic_bp_entry.get()),
                        "Diastolic": int(diastolic_bp_entry.get())
                    },
                    "Heart Rate": int(heart_rate_entry.get()),
                    "BMI": float(bmi_entry.get()),
                    "Triglycerides": int(triglycerides_entry.get())
                }
                db["health"].insert_one(health_data)
                whiteboard.insert(tk.END, "Health data inserted successfully.\n")
            except ValueError:
                whiteboard.insert(tk.END, "Invalid input. Please enter valid numbers.\n")

    def read_health_data():
        patient_id = patient_id_entry.get()
        health = db["health"].find_one({"Patient ID": patient_id})
        if health:
            whiteboard.insert(tk.END, f"Health Data: {str(health)}\n")
        else:
            whiteboard.insert(tk.END, "Health data not found.\n")

    def update_health_data():
        patient_id = patient_id_entry.get()
        health = db["health"].find_one({"Patient ID": patient_id})
        if health:
            update_data = {
                "$set": {
                    "Cholesterol": int(cholesterol_entry.get()),
                    "Blood Pressure": {
                        "Systolic": int(systolic_bp_entry.get()),
                        "Diastolic": int(diastolic_bp_entry.get())
                    },
                    "Heart Rate": int(heart_rate_entry.get()),
                    "BMI": float(bmi_entry.get()),
                    "Triglycerides": int(triglycerides_entry.get())
                }
            }
            db["health"].update_one({"Patient ID": patient_id}, update_data)
            whiteboard.insert(tk.END, "Health data updated successfully.\n")
        else:
            whiteboard.insert(tk.END, "Health data not found.\n")

    def delete_health_data():
        patient_id = patient_id_entry.get()
        result = db["health"].delete_one({"Patient ID": patient_id})
        if result.deleted_count == 1:
            whiteboard.insert(tk.END, "Health data deleted successfully.\n")
        else:
            whiteboard.insert(tk.END, "Health data not found.\n")

    def create_index_and_query():
        try:
            # Single Field Indexes for patient table
            field = "Triglycerides"
            index_name = f"{field}_index"
            index_info = db["health"].index_information()
            
            if index_name not in index_info:
                db["health"].create_index([(field, 1)], name=index_name)
                whiteboard.insert(tk.END, f"Index {index_name} created on field {field} in ascending order.\n")
            else:
                whiteboard.insert(tk.END, f"Index {index_name} already exists on field {field}.\n")
    
            # Perform the query using .hint() to use the "Age" index
            results = db["health"].find().hint({field: 1}).sort("Triglycerides", 1)
            whiteboard.insert(tk.END, "Query Results using hint on 'Triglycerides':\n")
            for result in results:
                whiteboard.insert(tk.END, f"{result}\n")
        except Exception as e:
            whiteboard.insert(tk.END, f"Error creating index or performing query with hint: {str(e)}\n")

    def drop_single_field_indexes():
        try:
            # Drop Single Field Indexes for patient table
            for field in ["Triglycerides"]:
                index_name = f"{field}_index"
                if index_name in db["health"].index_information():
                    db["health"].drop_index(index_name)
                    whiteboard.insert(tk.END, f"Index {index_name} dropped from field {field}.\n")
                else:
                    whiteboard.insert(tk.END, f"Index {index_name} not found on field {field}.\n")
        except Exception as e:
            whiteboard.insert(tk.END, f"Error dropping indexes: {str(e)}\n")   

    def find_patients_with_high_cholesterol():
        try:
            pipeline = [{"$match": {"Cholesterol": {"$gte": "200"}}}]  # Assuming Cholesterol is stored as an integer
            result = list(db["health"].aggregate(pipeline))
            if result:
                whiteboard.insert(tk.END, "Patients with Cholesterol greater than 200:\n")
                for patient in result:
                    whiteboard.insert(tk.END, f"Patient ID: {patient['Patient ID']}, Cholesterol: {patient['Cholesterol']}\n")
            else:
                whiteboard.insert(tk.END, "No patients found with Cholesterol greater than 200.\n")
        except Exception as e:
            whiteboard.insert(tk.END, f"Error finding patients with high cholesterol: {str(e)}\n")

    def calculate_avg_heart_rate():
        try:
            heart_rates = []
            for record in db["health"].find({}, {"Heart Rate": 1}):
                heart_rate = record.get("Heart Rate")
                if isinstance(heart_rate, int):
                    heart_rates.append(heart_rate)
                elif isinstance(heart_rate, str) and heart_rate.isdigit():
                    heart_rates.append(int(heart_rate))
            if heart_rates:
                avg_heart_rate = sum(heart_rates) / len(heart_rates)
                whiteboard.insert(tk.END, f"Average Heart Rate: {avg_heart_rate:.2f}\n")
            else:
                whiteboard.insert(tk.END, "No heart rate data available.\n")
        except Exception as e:
            whiteboard.insert(tk.END, f"Error calculating average heart rate: {str(e)}\n")

    def go_back():
        health_window.destroy()
        root.deiconify()


    # Health Data Management widgets and functions
    # Insert your existing Health Data Management code here, for example:
    tk.Label(health_window, text="Patient ID:").grid(row=0, column=0, padx=5, pady=5)
    tk.Label(health_window, text="Cholesterol:").grid(row=1, column=0, padx=5, pady=5)
    tk.Label(health_window, text="Systolic BP:").grid(row=2, column=0, padx=5, pady=5)
    tk.Label(health_window, text="Diastolic BP:").grid(row=3, column=0, padx=5, pady=5)
    tk.Label(health_window, text="Heart Rate:").grid(row=4, column=0, padx=5, pady=5)
    tk.Label(health_window, text="BMI:").grid(row=5, column=0, padx=5, pady=5)
    tk.Label(health_window, text="Triglycerides:").grid(row=6, column=0, padx=5, pady=5)

    patient_id_entry = tk.Entry(health_window)
    patient_id_entry.grid(row=0, column=1, padx=5, pady=5)
    cholesterol_entry = tk.Entry(health_window)
    cholesterol_entry.grid(row=1, column=1, padx=5, pady=5)
    systolic_bp_entry = tk.Entry(health_window)
    systolic_bp_entry.grid(row=2, column=1, padx=5, pady=5)
    diastolic_bp_entry = tk.Entry(health_window)
    diastolic_bp_entry.grid(row=3, column=1, padx=5, pady=5)
    heart_rate_entry = tk.Entry(health_window)
    heart_rate_entry.grid(row=4, column=1, padx=5, pady=5)
    bmi_entry = tk.Entry(health_window)
    bmi_entry.grid(row=5, column=1, padx=5, pady=5)
    triglycerides_entry = tk.Entry(health_window)
    triglycerides_entry.grid(row=6, column=1, padx=5, pady=5)

    insert_button = tk.Button(health_window, text="Insert", command=lambda: [insert_health_data(), insert_button.config(bg="blue", fg="white")])
    insert_button.grid(row=7, column=0, padx=5, pady=5)
    
    read_button = tk.Button(health_window, text="Read", command=lambda: [read_health_data(), read_button.config(bg="blue", fg="white")])
    read_button.grid(row=7, column=1, padx=5, pady=5)
    
    update_button = tk.Button(health_window, text="Update", command=lambda: [update_health_data(), update_button.config(bg="blue", fg="white")])
    update_button.grid(row=8, column=0, padx=5, pady=5)
    
    delete_button = tk.Button(health_window, text="Delete", command=lambda: [delete_health_data(), delete_button.config(bg="blue", fg="white")])
    delete_button.grid(row=8, column=1, padx=5, pady=5)
    
    index_button = tk.Button(health_window, text="Create Triglycerides Index", command=lambda: [create_index_and_query(), index_button.config(bg="blue",fg="white")])
    index_button.grid(row=9, column=0, padx=5, pady=5)
    
    drop_button = tk.Button(health_window, text="Drop Triglycerides Index", command=lambda: [drop_single_field_indexes(), drop_button.config(bg="blue", fg="white")])
    drop_button.grid(row=9, column=1, padx=5, pady=5)
    
    find_high_cholesterol_button = tk.Button(health_window, text="Find Patients with High Cholesterol", command=lambda: [find_patients_with_high_cholesterol(), find_high_cholesterol_button.config(bg="blue", fg="white")])
    find_high_cholesterol_button.grid(row=10, column=0, padx=5, pady=5)
    
    heartRate = tk.Button(health_window, text="AVE_heartRate", command=lambda: [calculate_avg_heart_rate(), heartRate.config(bg="blue", fg="white")])
    heartRate.grid(row=10, column=1, padx=5, pady=5)
    
    back_button = tk.Button(health_window, text="Back to Main Menu", command=go_back)
    back_button.grid(row=11, column=0, columnspan=2, pady=10)

    whiteboard_frame = tk.Frame(health_window)
    whiteboard_frame.grid(row=0, column=2, rowspan=11, padx=20, pady=5, sticky=tk.N+tk.S)

    whiteboard_scrollbar = tk.Scrollbar(whiteboard_frame)
    whiteboard_scrollbar.pack(side=tk.RIGHT, fill=tk.Y)

    global whiteboard
    whiteboard = tk.Text(whiteboard_frame, height=30, width=100, yscrollcommand=whiteboard_scrollbar.set)
    whiteboard.pack(side=tk.LEFT, fill=tk.BOTH)
    whiteboard_scrollbar.config(command=whiteboard.yview)

    root.withdraw()
    health_window.mainloop()


#############################################  Health History  #################################################################
def open_health_history_data_management():
    health_history_window = tk.Toplevel(root)
    health_history_window.title("Health History Data Management")

    # Function to insert health history data
    def insert_health_history_data():
        patient_id = patient_id_entry.get()
    
        # Check if patient ID exists in the patient collection
        existing_patient = db["healthHistory"].find_one({"Patient ID": patient_id})
    
        if existing_patient:
            whiteboard.insert(tk.END, "Patient ID already exists. Please use a different ID.\n")
        else:
            try:
                health_history_data = {
                    "Patient ID": patient_id_entry.get(),
                    "Diabetes": int(diabetes_entry.get()),
                    "Family History": int(family_history_entry.get()),
                    "Previous Heart Problems": int(previous_heart_problems_entry.get()),
                    "Medication Use": int(medication_use_entry.get()),
                    "Stress Level": int(stress_level_entry.get()),
                    "Income": float(income_entry.get()),
                    "Heart Attack Risk": int(heart_attack_risk_entry.get())
                }
                db["healthHistory"].insert_one(health_history_data)
                whiteboard.insert(tk.END, "Health History data inserted successfully.\n")
            except ValueError:
                whiteboard.insert(tk.END, "Invalid input. Please enter valid numbers.\n")
    
    # Function to read health history data
    def read_health_history_data():
        try:
            patient_id = patient_id_entry.get()
            health_history = db["healthHistory"].find_one({"Patient ID": patient_id})
            if health_history:
                whiteboard.insert(tk.END, f"Health History Data: {str(health_history)}\n")
            else:
                whiteboard.insert(tk.END, "Health history data not found.\n")
        except Exception as e:
            whiteboard.insert(tk.END, f"Error reading health history data: {str(e)}\n")

    # Function to update health history data
    def update_health_history_data():
        try:
            patient_id = patient_id_entry.get()
            health_history = db["healthHistory"].find_one({"Patient ID": patient_id})
            if health_history:
                update_data = {
                    "$set": {
                        "Diabetes": int(diabetes_entry.get()),
                        "Family History": int(family_history_entry.get()),
                        "Previous Heart Problems": int(previous_heart_problems_entry.get()),
                        "Medication Use": int(medication_use_entry.get()),
                        "Stress Level": int(stress_level_entry.get()),
                        "Income": float(income_entry.get()),
                        "Heart Attack Risk": int(heart_attack_risk_entry.get())
                    }
                }
                db["healthHistory"].update_one({"Patient ID": patient_id}, update_data)
                whiteboard.insert(tk.END, "Health history data updated successfully.\n")
            else:
                whiteboard.insert(tk.END, "Health history data not found.\n")
        except Exception as e:
            whiteboard.insert(tk.END, f"Error updating health history data: {str(e)}\n")
    
    # Function to delete health history data
    def delete_health_history_data():
        try:
            patient_id = patient_id_entry.get()
            result = db["healthHistory"].delete_one({"Patient ID": patient_id})
            if result.deleted_count == 1:
                whiteboard.insert(tk.END, "Health history data deleted successfully.\n")
            else:
                whiteboard.insert(tk.END, "Health history data not found.\n")
        except Exception as e:
            whiteboard.insert(tk.END, f"Error deleting health history data: {str(e)}\n")

    def create_compound_indexes():
        try:
            # Compound Indexes for healthHistory table
            compound_index_name = "Diabetes_FamilyHistory_index"
            index_info = db["healthHistory"].index_information()
    
            if compound_index_name not in index_info:
                db["healthHistory"].create_index([("Diabetes", 1), ("Family History", 1)], name=compound_index_name)
                whiteboard.insert(tk.END, "Compound indexes created successfully.\n")
            else:
                whiteboard.insert(tk.END, f"Compound index {compound_index_name} already exists.\n")
    
            # Display index information
            # index_info = db["healthHistory"].index_information()
            # whiteboard.insert(tk.END, "Indexes:\n")
            # for index_name, info in index_info.items():
            #     whiteboard.insert(tk.END, f"{index_name}: {info}\n")
            
            # Perform a query using the compound index with a hint
            results = db["healthHistory"].find({"Diabetes": {"$exists": True}}).hint([("Diabetes", 1), ("Family History", 1)])
            whiteboard.insert(tk.END, "Query results using compound index:\n")
            for result in results:
                whiteboard.insert(tk.END, f"{result}\n")
        except Exception as e:
            whiteboard.insert(tk.END, f"Error creating indexes or querying: {str(e)}\n")

    def drop_compound_indexes():
        try:
            # List all indexes in the collection
            indexes = db["healthHistory"].index_information()
            index_name_to_drop = None
            
            # Check if the desired compound index exists
            for index_name, index_info in indexes.items():
                if index_info['key'] == [('Diabetes', 1), ('Family History', 1)]:
                    index_name_to_drop = index_name
                    break
            
            if index_name_to_drop:
                # Drop the compound index by name
                db["healthHistory"].drop_index(index_name_to_drop)
                whiteboard.insert(tk.END, "Compound index dropped successfully.\n")
            else:
                whiteboard.insert(tk.END, "Compound index not found.\n")
        except Exception as e:
            whiteboard.insert(tk.END, f"Error dropping compound indexes: {str(e)}\n")

    # Function to count patients with previous heart problems
    def count_patients_with_previous_heart_problems():
        try:
            pipeline = [{"$match": {"Previous Heart Problems": "1"}}, {"$count": "count"}]
            result = list(db["healthHistory"].aggregate(pipeline))
            if result:
                count = result[0]["count"]
                whiteboard.insert(tk.END, f"Number of patients with previous heart problems: {count}\n")
            else:
                whiteboard.insert(tk.END, "No patients found with previous heart problems.\n")
        except Exception as e:
            whiteboard.insert(tk.END, f"Error counting patients with previous heart problems: {str(e)}\n")
    
    def calculate_avg_stress_level():
        try:
            stress_levels = []
            for record in db["healthHistory"].find({}, {"Stress Level": 1}):
                stress_level = record.get("Stress Level")
                if isinstance(stress_level, int):
                    stress_levels.append(stress_level)
                elif isinstance(stress_level, str) and stress_level.isdigit():
                    stress_levels.append(int(stress_level))
            if stress_levels:
                avg_stress_level = sum(stress_levels) / len(stress_levels)
                whiteboard.insert(tk.END, f"Average Stress Level: {avg_stress_level:.2f}\n")
            else:
                whiteboard.insert(tk.END, "No Stress Level data available.\n")
        except Exception as e:
            whiteboard.insert(tk.END, f"Error calculating average stress level: {str(e)}\n")

    # Function to select smokers
    def show_selected_patients_with_diabetes():
        try:
            pipeline = [
                {"$match": {"Diabetes": "1"}}
            ]
            result = list(db["healthHistory"].aggregate(pipeline))
            if result:
                for diabetes in result:
                    whiteboard.insert(tk.END, f"diabetes: {str(diabetes)}\n")
            else:
                whiteboard.insert(tk.END, "No diabetes found.\n")
        except Exception as e:
            whiteboard.insert(tk.END, f"Error selecting diabetes: {str(e)}\n") 
    
    # Function to select patients with previous heart problems
    def select_patients_with_previous_heart_problems():
        try:
            pipeline = [
                {"$match": {"Previous Heart Problems": "0"}}
            ]
            result = list(db["healthHistory"].aggregate(pipeline))
            if result:
                whiteboard.insert(tk.END, "Patients not have Previous Heart Problems:\n")
                for patient in result:
                    whiteboard.insert(tk.END, f"{str(patient)}\n")
            else:
                whiteboard.insert(tk.END, "No patients found with previous heart problems.\n")
        except Exception as e:
            whiteboard.insert(tk.END, f"Error selecting patients with previous heart problems: {str(e)}\n")

    def go_back():
        health_history_window.destroy()
        root.deiconify()

    # Health Data Management widgets and functions
    # Insert your existing Health Data Management code here, for example:
    tk.Label(health_history_window, text="Patient ID:").grid(row=0, column=0, padx=5, pady=5)
    tk.Label(health_history_window, text="Diabetes (0/1):").grid(row=1, column=0, padx=5, pady=5)
    tk.Label(health_history_window, text="Family History (0/1):").grid(row=2, column=0, padx=5, pady=5)
    tk.Label(health_history_window, text="Previous Heart Problems (0/1):").grid(row=3, column=0, padx=5, pady=5)
    tk.Label(health_history_window, text="Medication Use (0/1):").grid(row=4, column=0, padx=5, pady=5)
    tk.Label(health_history_window, text="Stress Level (0-10):").grid(row=5, column=0, padx=5, pady=5)
    tk.Label(health_history_window, text="Income:").grid(row=6, column=0, padx=5, pady=5)
    tk.Label(health_history_window, text="Heart Attack Risk (0/1):").grid(row=7, column=0, padx=5, pady=5)

    # Entry fields
    patient_id_entry = tk.Entry(health_history_window)
    patient_id_entry.grid(row=0, column=1, padx=5, pady=5)
    diabetes_entry = tk.Entry(health_history_window)
    diabetes_entry.grid(row=1, column=1, padx=5, pady=5)
    family_history_entry = tk.Entry(health_history_window)
    family_history_entry.grid(row=2, column=1, padx=5, pady=5)
    previous_heart_problems_entry = tk.Entry(health_history_window)
    previous_heart_problems_entry.grid(row=3, column=1, padx=5, pady=5)
    medication_use_entry = tk.Entry(health_history_window)
    medication_use_entry.grid(row=4, column=1, padx=5, pady=5)
    stress_level_entry = tk.Entry(health_history_window)
    stress_level_entry.grid(row=5, column=1, padx=5, pady=5)
    income_entry = tk.Entry(health_history_window)
    income_entry.grid(row=6, column=1, padx=5, pady=5)
    heart_attack_risk_entry = tk.Entry(health_history_window)
    heart_attack_risk_entry.grid(row=7, column=1, padx=5, pady=5)

    # Buttons
    insert_button = tk.Button(health_history_window, text="Insert", command=lambda: [insert_health_history_data(), insert_button.config(bg="blue", fg="white")])
    insert_button.grid(row=8, column=0, padx=5, pady=5)
    
    read_button = tk.Button(health_history_window, text="Read", command=lambda: [read_health_history_data(), read_button.config(bg="blue", fg="white")])
    read_button.grid(row=8, column=1, padx=5, pady=5)
    
    update_button = tk.Button(health_history_window, text="Update", command=lambda: [update_health_history_data(), update_button.config(bg="blue", fg="white")])
    update_button.grid(row=9, column=0, padx=5, pady=5)
    
    delete_button = tk.Button(health_history_window, text="Delete", command=lambda: [delete_health_history_data(), delete_button.config(bg="blue", fg="white")])
    delete_button.grid(row=9, column=1, padx=5, pady=5)
    
    index_button = tk.Button(health_history_window, text="Create Compound Indexes Di FH", command=lambda: [create_compound_indexes(), index_button.config(bg="blue", fg="white")])
    index_button.grid(row=10, column=0, padx=5, pady=5)
    
    drop_index_button = tk.Button(health_history_window, text="drop_compound_indexes", command=lambda: [drop_compound_indexes(), drop_index_button.config(bg="blue", fg="white")])
    drop_index_button.grid(row=10, column=1, padx=5, pady=5)
    
    count_previous_heart_problems_button = tk.Button(health_history_window, text="Count Patients with Previous Heart Problems", command=lambda: [count_patients_with_previous_heart_problems(), count_previous_heart_problems_button.config(bg="blue", fg="white")])
    count_previous_heart_problems_button.grid(row=11, column=0, padx=5, pady=5)
    
    calculate_avg_stress_level_button = tk.Button(health_history_window, text="Calculate Average Stress Level", command=lambda: [calculate_avg_stress_level(), calculate_avg_stress_level_button.config(bg="blue", fg="white")])
    calculate_avg_stress_level_button.grid(row=11, column=1, padx=5, pady=5)
    
    show_selected_button = tk.Button(health_history_window, text="Select Patients with Diabetes", command=lambda: [show_selected_patients_with_diabetes(), show_selected_button.config(bg="blue", fg="white")])
    show_selected_button.grid(row=12, column=0, padx=5, pady=5)
    
    select_previous_heart_problems_button = tk.Button(health_history_window, text="Select Patients not have Previous Heart Problems", command=lambda: [select_patients_with_previous_heart_problems(), select_previous_heart_problems_button.config(bg="blue", fg="white")])
    select_previous_heart_problems_button.grid(row=12, column=1, padx=5, pady=5)

    back_button = tk.Button(health_history_window, text="Back to Main Menu", command=go_back)
    back_button.grid(row=13, column=0, columnspan=2, pady=10)

    whiteboard_frame = tk.Frame(health_history_window)
    whiteboard_frame.grid(row=0, column=2, rowspan=11, padx=20, pady=5, sticky=tk.N+tk.S)

    whiteboard_scrollbar = tk.Scrollbar(whiteboard_frame)
    whiteboard_scrollbar.pack(side=tk.RIGHT, fill=tk.Y)

    global whiteboard
    whiteboard = tk.Text(whiteboard_frame, height=30, width=100, yscrollcommand=whiteboard_scrollbar.set)
    whiteboard.pack(side=tk.LEFT, fill=tk.BOTH)
    whiteboard_scrollbar.config(command=whiteboard.yview)

    root.withdraw()
    health_history_window.mainloop()

#############################################  Lifestyle  #################################################################
def open_lifestyle_data_management():
    lifestyle_window = tk.Toplevel(root)
    lifestyle_window.title("Lifestyle Data Management")

    # Function to insert lifestyle data

    def insert_lifestyle_data():
        patient_id = patient_id_entry.get()
    
        # Check if patient ID exists in the patient collection
        existing_patient = db["lifestyle"].find_one({"Patient ID": patient_id})
    
        if existing_patient:
            whiteboard.insert(tk.END, "Patient ID already exists. Please use a different ID.\n")
        else:
            try:
                lifestyle_data = {
                    "Patient ID": patient_id_entry.get(),
                    "Smoking": int(smoking_entry.get()),
                    "Alcohol Consumption": int(alcohol_entry.get()),
                    "Exercise Hours Per Week": float(exercise_entry.get()),
                    "Diet": diet_entry.get(),
                    "Sedentary Hours Per Day": float(sedentary_entry.get()),
                    "Physical Activity Days Per Week": int(physical_activity_entry.get()),
                    "Sleep Hours Per Day": float(sleep_entry.get())
                }
                db["lifestyle"].insert_one(lifestyle_data)
                whiteboard.insert(tk.END, "Lifestyle data inserted successfully.\n")
            except ValueError:
                whiteboard.insert(tk.END, "Invalid input. Please enter valid numbers.\n")
            except Exception as e:
                whiteboard.insert(tk.END, f"Error inserting lifestyle data: {str(e)}\n")

    # Function to read lifestyle data
    def read_lifestyle_data():
        try:
            patient_id = patient_id_entry.get()
            lifestyle = db["lifestyle"].find_one({"Patient ID": patient_id})
            if lifestyle:
                whiteboard.insert(tk.END, f"Lifestyle Data: {str(lifestyle)}\n")
            else:
                whiteboard.insert(tk.END, "Lifestyle data not found.\n")
        except Exception as e:
            whiteboard.insert(tk.END, f"Error reading lifestyle data: {str(e)}\n")

    # Function to update lifestyle data
    def update_lifestyle_data():
        try:
            patient_id = patient_id_entry.get()
            lifestyle = db["lifestyle"].find_one({"Patient ID": patient_id})
            if lifestyle:
                update_data = {
                    "$set": {
                        "Smoking": int(smoking_entry.get()),
                        "Alcohol Consumption": int(alcohol_entry.get()),
                        "Exercise Hours Per Week": float(exercise_entry.get()),
                        "Diet": diet_entry.get(),
                        "Sedentary Hours Per Day": float(sedentary_entry.get()),
                        "Physical Activity Days Per Week": int(physical_activity_entry.get()),
                        "Sleep Hours Per Day": float(sleep_entry.get())
                    }
                }
                db["lifestyle"].update_one({"Patient ID": patient_id}, update_data)
                whiteboard.insert(tk.END, "Lifestyle data updated successfully.\n")
            else:
                whiteboard.insert(tk.END, "Lifestyle data not found.\n")
        except Exception as e:
            whiteboard.insert(tk.END, f"Error updating lifestyle data: {str(e)}\n")

    # Function to delete lifestyle data
    def delete_lifestyle_data():
        try:
            patient_id = patient_id_entry.get()
            result = db["lifestyle"].delete_one({"Patient ID": patient_id})
            if result.deleted_count == 1:
                whiteboard.insert(tk.END, "Lifestyle data deleted successfully.\n")
            else:
                whiteboard.insert(tk.END, "Lifestyle data not found.\n")
        except Exception as e:
            whiteboard.insert(tk.END, f"Error deleting lifestyle data: {str(e)}\n")

    # Function to create index on "Sleep Hours Per Day"
    def create_sleep_index():
        try:
            field = "Sleep Hours Per Day"
            index_name = f"{field.replace(' ', '_')}_index"
            index_info = db["lifestyle"].index_information()
    
            # Check if the field already has an index, regardless of its name
            existing_index = False
            for name, info in index_info.items():
                if field in dict(info['key']).keys():
                    existing_index = True
                    whiteboard.insert(tk.END, f"Index {name} already exists on field {field}.\n")
                    break
    
            # Create the index if it does not exist
            if not existing_index:
                db["lifestyle"].create_index([(field, 1)], name=index_name)
                whiteboard.insert(tk.END, f"Index {index_name} created on field {field} in ascending order.\n")
    
            # Perform the query using .hint() to use the "Sleep Hours Per Day" index
            results = db["lifestyle"].find().hint({field: 1}).sort("Sleep Hours Per Day", -1)
            whiteboard.insert(tk.END, "Query Results using hint on 'Sleep Hours Per Day':\n")
            for result in results:
                whiteboard.insert(tk.END, f"{result}\n")
        except Exception as e:
            whiteboard.insert(tk.END, f"Error creating index or performing query with hint: {str(e)}\n")

    # Function to drop index on "Sleep Hours Per Day"
    def drop_sleep_index():
        try:
            field = "Sleep Hours Per Day"
            index_name = f"{field.replace(' ', '_')}_index"
            index_info = db["lifestyle"].index_information()
    
            # Find the index to drop
            index_to_drop = None
            for name, info in index_info.items():
                if field in dict(info['key']).keys():
                    index_to_drop = name
                    break
    
            # Drop the index if it exists
            if index_to_drop:
                db["lifestyle"].drop_index(index_to_drop)
                whiteboard.insert(tk.END, f"Index {index_to_drop} dropped from field {field}.\n")
            else:
                whiteboard.insert(tk.END, f"No index found on field {field} to drop.\n")
        except Exception as e:
            whiteboard.insert(tk.END, f"Error dropping index: {str(e)}\n")

    # Function to count patients with unhealthy diet
    def count_patients_with_unhealthy_diet():
        try:
            pipeline = [{"$match": {"Diet": "Unhealthy"}}, {"$count": "count"}]
            result = list(db["lifestyle"].aggregate(pipeline))
            if result:
                count = result[0]["count"]
                whiteboard.insert(tk.END, f"Number of patients with unhealthy diet: {count}\n")
            else:
                whiteboard.insert(tk.END, "No patients found with unhealthy diet.\n")
        except Exception as e:
            whiteboard.insert(tk.END, f"Error counting patients with unhealthy diet: {str(e)}\n")

    # Function to count patients with regular exercise
    def count_patients_with_regular_exercise():
        try:
            pipeline = [
                {"$match": {"Exercise Hours Per Week": {"$gte": "3"}}}, {"$count": "count"}
            ]
            result = list(db["lifestyle"].aggregate(pipeline))
            if result:
                count = result[0]["count"]
                whiteboard.insert(tk.END, f"Number of patients with regular exercise: {count}\n")
            else:
                whiteboard.insert(tk.END, "No patients found with regular exercise.\n")
        except Exception as e:
            whiteboard.insert(tk.END, f"Error counting patients with regular exercise: {str(e)}\n")

    # Function to select smokers
    def select_smokers():
        try:
            pipeline = [
                {"$match": {"Smoking": "1"}}
            ]
            result = list(db["lifestyle"].aggregate(pipeline))
            if result:
                for smoker in result:
                    whiteboard.insert(tk.END, f"Smoker: {str(smoker)}\n")
            else:
                whiteboard.insert(tk.END, "No smokers found.\n")
        except Exception as e:
            whiteboard.insert(tk.END, f"Error selecting smokers: {str(e)}\n")

    # Function to perform map-reduce operation for diet categories
    def map_reduce_diet_categories():
        try:
            # Define the map function
            mapper = Code("""
                function () {
                    if (this.Diet) {
                        emit(this.Diet, 1);
                    }
                }
            """)
    
            # Define the reduce function
            reducer = Code("""
                function (key, values) {
                    return Array.sum(values);
                }
            """)
    
            # Execute the MapReduce operation
            result = db.command('mapReduce', "lifestyle", map=mapper, reduce=reducer, out={'inline': 1})
    
            # Print results
            whiteboard.insert(tk.END, "MapReduce Results for Diet Categories:\n")
            if result['results']:
                for doc in result['results']:
                    whiteboard.insert(tk.END, f"Diet: {doc['_id']}, Count: {doc['value']}\n")
            else:
                whiteboard.insert(tk.END, "No results found. Please check if the 'diet' field exists and has valid values.\n")
        except Exception as e:
            whiteboard.insert(tk.END, f"Error performing MapReduce operation: {str(e)}\n")
            
    def go_back():
        lifestyle_window.destroy()
        root.deiconify()

    # Health Data Management widgets and functions
    # Insert your existing Health Data Management code here, for example:
    # Labels
    tk.Label(lifestyle_window, text="Patient ID:").grid(row=0, column=0, padx=5, pady=5)
    tk.Label(lifestyle_window, text="Smoking (0/1):").grid(row=1, column=0, padx=5, pady=5)
    tk.Label(lifestyle_window, text="Alcohol Consumption (0/1):").grid(row=2, column=0, padx=5, pady=5)
    tk.Label(lifestyle_window, text="Exercise Hours Per Week:").grid(row=3, column=0, padx=5, pady=5)
    tk.Label(lifestyle_window, text="Diet(Healthy,Average,Unhealthy):").grid(row=4, column=0, padx=5, pady=5)
    tk.Label(lifestyle_window, text="Sedentary Hours Per Day:").grid(row=5, column=0, padx=5, pady=5)
    tk.Label(lifestyle_window, text="Physical Activity Days Per Week:").grid(row=6, column=0, padx=5, pady=5)
    tk.Label(lifestyle_window, text="Sleep Hours Per Day:").grid(row=7, column=0, padx=5, pady=5)

    # Entry fields
    patient_id_entry = tk.Entry(lifestyle_window)
    patient_id_entry.grid(row=0, column=1, padx=5, pady=5)
    smoking_entry = tk.Entry(lifestyle_window)
    smoking_entry.grid(row=1, column=1, padx=5, pady=5)
    alcohol_entry = tk.Entry(lifestyle_window)
    alcohol_entry.grid(row=2, column=1, padx=5, pady=5)
    exercise_entry = tk.Entry(lifestyle_window)
    exercise_entry.grid(row=3, column=1, padx=5, pady=5)
    diet_entry = tk.Entry(lifestyle_window)
    diet_entry.grid(row=4, column=1, padx=5, pady=5)
    sedentary_entry = tk.Entry(lifestyle_window)
    sedentary_entry.grid(row=5, column=1, padx=5, pady=5)
    physical_activity_entry = tk.Entry(lifestyle_window)
    physical_activity_entry.grid(row=6, column=1, padx=5, pady=5)
    sleep_entry = tk.Entry(lifestyle_window)
    sleep_entry.grid(row=7, column=1, padx=5, pady=5)

    #Button
    insert_button = tk.Button(lifestyle_window, text="Insert", command=lambda: [insert_lifestyle_data(), insert_button.config(bg="blue", fg="white")])
    insert_button.grid(row=8, column=0, padx=5, pady=5)
    
    read_button = tk.Button(lifestyle_window, text="Read", command=lambda: [read_lifestyle_data(), read_button.config(bg="blue", fg="white")])
    read_button.grid(row=8, column=1, padx=5, pady=5)
    
    update_button = tk.Button(lifestyle_window, text="Update", command=lambda: [update_lifestyle_data(), update_button.config(bg="blue", fg="white")])
    update_button.grid(row=9, column=0, padx=5, pady=5)
    
    delete_button = tk.Button(lifestyle_window, text="Delete", command=lambda: [delete_lifestyle_data(), delete_button.config(bg="blue", fg="white")])
    delete_button.grid(row=9, column=1, padx=5, pady=5)
    
    index_button = tk.Button(lifestyle_window, text="Create Index Sleep Hours", command=lambda: [create_sleep_index(), index_button.config(bg="blue", fg="white")])
    index_button.grid(row=10, column=0, padx=5, pady=5)
    
    drop_index_button = tk.Button(lifestyle_window, text="Drop Index", command=lambda: [drop_sleep_index(), drop_index_button.config(bg="blue", fg="white")])
    drop_index_button.grid(row=10, column=1, padx=5, pady=5)
    
    count_unhealthy_button = tk.Button(lifestyle_window, text="Count Patients with Unhealthy Diet", command=lambda: [count_patients_with_unhealthy_diet(), count_unhealthy_button.config(bg="blue", fg="white")])
    count_unhealthy_button.grid(row=11, column=0, padx=5, pady=5)
    
    count_exercise_button = tk.Button(lifestyle_window, text="Count Patients with Regular Exercise", command=lambda: [count_patients_with_regular_exercise(), count_exercise_button.config(bg="blue", fg="white")])
    count_exercise_button.grid(row=11, column=1, padx=5, pady=5)
    
    select_smokers_button = tk.Button(lifestyle_window, text="Select Smokers", command=lambda: [select_smokers(), select_smokers_button.config(bg="blue", fg="white")])
    select_smokers_button.grid(row=12, column=0, padx=5, pady=5)
    
    map_reduce_diet_button = tk.Button(lifestyle_window, text="MapReduce Diet Categories", command=lambda: [map_reduce_diet_categories(), map_reduce_diet_button.config(bg="blue", fg="white")])
    map_reduce_diet_button.grid(row=12, column=1, padx=5, pady=5)

    back_button = tk.Button(lifestyle_window, text="Back to Main Menu", command=go_back)
    back_button.grid(row=13, column=0, columnspan=2, pady=10)

    whiteboard_frame = tk.Frame(lifestyle_window)
    whiteboard_frame.grid(row=0, column=2, rowspan=11, padx=20, pady=5, sticky=tk.N+tk.S)

    whiteboard_scrollbar = tk.Scrollbar(whiteboard_frame)
    whiteboard_scrollbar.pack(side=tk.RIGHT, fill=tk.Y)

    global whiteboard
    whiteboard = tk.Text(whiteboard_frame, height=30, width=100, yscrollcommand=whiteboard_scrollbar.set)
    whiteboard.pack(side=tk.LEFT, fill=tk.BOTH)
    whiteboard_scrollbar.config(command=whiteboard.yview)

    root.withdraw()
    lifestyle_window.mainloop()

patient_button = tk.Button(root, text="Patient Data Management", command=open_patient_data_management)
patient_button.grid(row=0, column=0, padx=10, pady=10)

health_button = tk.Button(root, text="Health Data Management", command=open_health_data_management)
health_button.grid(row=1, column=0, padx=10, pady=10)

health_button = tk.Button(root, text="Health History Data Management", command=open_health_history_data_management)
health_button.grid(row=2, column=0, padx=10, pady=10)

health_button = tk.Button(root, text="Lifestyle Data Management", command=open_lifestyle_data_management)
health_button.grid(row=3, column=0, padx=10, pady=10)

root.mainloop()
                        